In [ ]:
import re
from requests import get
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from wordsegment import load, segment
load()
stop_words = set(stopwords.words('english'))


#function for normalizing the text data    
#input: string; output: string
def processed(phrase):
    lemmatizer = WordNetLemmatizer() 
    phrase=phrase.lower()
    phrase=' '.join(segment(phrase))
    phrase = re.sub('[^A-Za-z0-9]+', '  ', phrase)
    word_tokens = word_tokenize(phrase)
    filtered_sentence = [lemmatizer.lemmatize(w, pos='v') for w in word_tokens if not w in stop_words]
    filtered_sentence = [lemmatizer.lemmatize(w, pos='a') for w in filtered_sentence]
    filtered_sentence = [lemmatizer.lemmatize(w, pos='n') for w in filtered_sentence]
    phrase=' '.join(filtered_sentence)
    phrase = re.sub('[0-9]+', '', phrase)
    return phrase

def index_function(url):
    raw_data=[]
    raw = get(url) 
    raw_data.append(raw.content)
    content=[]
    soup = BeautifulSoup(raw_data[0], 'html.parser')
    for j in soup.find_all(['a','p','title','div','h1','h2','h3','h4']):
        content.append(j.get_text())
    processed_data=[]
    s=''
    for j in content:
        s=s+str(processed(j))
    processed_data.append(s)
    return(processed_data)

processed_data=index_function('https://www.olacabs.com/')

count_vect = CountVectorizer(ngram_range=(2,3))
sparse_matrix1=count_vect.fit_transform(processed_data)

sum_words = sparse_matrix1.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in count_vect.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
print(words_freq[0][0])